In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys

from utilities2015 import *

import time

from scipy.spatial.distance import cdist, pdist, squareform
from scipy.cluster.hierarchy import average, fcluster, single, complete

from joblib import Parallel, delayed

from skimage.color import gray2rgb
from skimage.measure import find_contours
from skimage.util import img_as_float

import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('/home/yuncong/project/opencv-2.4.9/release/lib/python2.7/site-packages')
import cv2

from networkx import from_dict_of_lists, Graph, adjacency_matrix, dfs_postorder_nodes
from networkx.algorithms import node_connected_component

In [4]:
stack = 'MD593'

In [5]:
os.environ['GORDON_DATA_DIR'] = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed'
os.environ['GORDON_REPO_DIR'] = '/oasis/projects/nsf/csd395/yuncong/Brain'
os.environ['GORDON_RESULT_DIR'] = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_results'

dm = DataManager(data_dir=os.environ['GORDON_DATA_DIR'], 
  repo_dir=os.environ['GORDON_REPO_DIR'], 
  result_dir=os.environ['GORDON_RESULT_DIR'], labeling_dir=os.environ['GORDON_LABELING_DIR'])

print 'reading image ...',b
t = time.time()
dm.set_image(stack, 100)
print 'done in', time.time() - t, 'seconds'

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/PIL/Image.py:2261: DecompressionBombWarning: Image size (203470848 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


reading image ... done in 38.843485117 seconds


In [55]:
neighbors = dm.load_pipeline_result('neighbors', 'pkl')

In [62]:
edge_neighbors = dm.load_pipeline_result('edgeNeighbors', 'pkl')

In [65]:
texton_map = dm.load_pipeline_result('texMap', 'npy')

AssertionError: 100: Pipeline result 'texMap.npy' does not exist

In [10]:
segmentation = dm.load_pipeline_result('segmentation', 'npy')

In [18]:
n_superpixels = segmentation.max() + 1

In [51]:
def diff_offset(x_offset, y_offset):

    h, w = segmentation.shape
    
    if x_offset == 1 and y_offset == -1:
        d = np.dstack([segmentation[0:h-1, 1:w], segmentation[1:h, 0:w-1]])
        # if diff's location y,x is (0,0), the edge is at (1,0) and (0,1)
        ys, xs = np.mgrid[:d.shape[0], :d.shape[1]]
        nzs = ~(d[...,0]==d[...,1])
        r = np.c_[d[nzs], ys[nzs] + 1, xs[nzs], ys[nzs], xs[nzs] + 1] # [sp_label1, sp_label2, y1,x1,y2,x2]        
    else:
        moving_x_low = max(x_offset, 0)
        moving_x_high = min(x_offset + w, w)
        moving_width = moving_x_high - moving_x_low
        moving_y_low = max(y_offset, 0)
        moving_y_high = min(y_offset + h, h)
        moving_height = moving_y_high - moving_y_low

        d = np.dstack([segmentation[moving_y_low:moving_y_high, moving_x_low:moving_x_high], 
                       segmentation[:moving_height, :moving_width]])

        ys, xs = np.mgrid[:d.shape[0], :d.shape[1]]
        nzs = ~(d[...,0]==d[...,1])
        # if diff's location y,x is (0,0), the edge is at (0,0) and (y_offset, x_offset)
        r = np.c_[d[nzs], ys[nzs], xs[nzs], ys[nzs] + y_offset, xs[nzs] + x_offset] # [sp_label1, sp_label2, y1,x1,y2,x2]     
    
    return r

In [52]:
diffs = np.vstack([diff_offset(1,0), diff_offset(0,1), diff_offset(1,1), diff_offset(1,-1)])

neighbors = [set() for _ in range(n_superpixels)]

for i, j, y1, x1, y2, x2 in diffs:
    if i != -1:
        neighbors[i].add(j)
    if j != -1:
        neighbors[j].add(i)
        